# RadGraph MIMIC-CXR

In [6]:
import json
import pprint

graph_train_path = '/DATA1/llm-research/RadGraph/physionet.org/files/radgraph/1.0.0/train.json'

with open(graph_train_path, 'r') as f:
    train_data = json.load(f)

# pprint.pprint(train_data)

entities = train_data['p18/p18004941/s58821758.txt']['entities']
pprint.pprint(entities)

# print text
print(train_data['p18/p18004941/s58821758.txt']['text'])


# Step 1: Initialize an empty list for tokens
all_tokens = []

# Step 2: Iterate through each document in train_data
for document_key in train_data:
    # Access the 'entities' for the current document
    entities = train_data[document_key]['entities']
    
    # Step 4: Iterate through each entity in 'entities'
    for entity_key in entities:
        # Step 5: Access the 'tokens' field and append its value to the all_tokens list
        all_tokens.append(entities[entity_key]['tokens'])

# At this point, all_tokens list contains all tokens from the entire train_data
print(len(all_tokens))
# print(all_tokens)

# Step 1: Initialize an empty set for tokens
all_tokens = set()

# Step 2: Iterate through each document in train_data
for document_key in train_data:
    # Access the 'entities' for the current document
    entities = train_data[document_key]['entities']
    
    # Step 4: Iterate through each entity in 'entities'
    for entity_key in entities:
        # Step 5: Access the 'tokens' field and add its value to the all_tokens set with 'label', as tupel
        all_tokens.add((entities[entity_key]['tokens'], entities[entity_key]['label']))

# Convert the set back to a list if you need a list format
unique_tokens_list = list(all_tokens)

# At this point, unique_tokens_list contains all unique tokens from the entire train_data
print(len(unique_tokens_list))
# print(unique_tokens_list)

{'1': {'end_ix': 36,
       'label': 'ANAT-DP',
       'relations': [],
       'start_ix': 36,
       'tokens': 'Lungs'},
 '2': {'end_ix': 38,
       'label': 'OBS-DP',
       'relations': [['located_at', '1']],
       'start_ix': 38,
       'tokens': 'clear'},
 '3': {'end_ix': 40,
       'label': 'OBS-DP',
       'relations': [['located_at', '4'],
                     ['located_at', '5'],
                     ['located_at', '7']],
       'start_ix': 40,
       'tokens': 'Normal'},
 '4': {'end_ix': 41,
       'label': 'ANAT-DP',
       'relations': [],
       'start_ix': 41,
       'tokens': 'cardiomediastinal'},
 '5': {'end_ix': 43,
       'label': 'ANAT-DP',
       'relations': [],
       'start_ix': 43,
       'tokens': 'hilar'},
 '6': {'end_ix': 44,
       'label': 'ANAT-DP',
       'relations': [['modify', '4'], ['modify', '5']],
       'start_ix': 44,
       'tokens': 'silhouettes'},
 '7': {'end_ix': 46,
       'label': 'ANAT-DP',
       'relations': [],
       'start_ix': 46,
  

In [5]:
unique_second_elements = {token[1] for token in all_tokens}
print(unique_second_elements)
num_unique_second_elements = len(unique_second_elements)
print(num_unique_second_elements)

type_counts = {}
for token in all_tokens:
    token_type = token[1]
    if token_type in type_counts:
        type_counts[token_type] += 1
    else:
        type_counts[token_type] = 1

print(type_counts)



{'OBS-U', 'OBS-DA', 'OBS-DP', 'ANAT-DP'}
4
{'OBS-DP': 812, 'ANAT-DP': 398, 'OBS-U': 190, 'OBS-DA': 153}


entity: 12388  
unique: 1250

In [ ]:
relations = set()

# Step 1: Iterate through each document in train_data
for document_key in train_data:
    # Access the 'entities' for the current document
    entities = train_data[document_key]['entities']
    
    # Step 2: Iterate through each entity in 'entities'
    for entity_key in entities:
        # Access the entity and its relations
        entity = entities[entity_key]
        entity_relations = entity['relations']
        
        # Step 3: Iterate through each relation in 'entity_relations'
        for relation in entity_relations:
            # Extract the related entity key and relation type
            related_entity_key = relation[1]
            relation_type = relation[0]
            
            # Get the related entity
            related_entity = entities[related_entity_key]
            
            # Create a tuple with entity1, entity2, and relation
            relation_tuple = (entity['tokens'], related_entity['tokens'], relation_type)
            
            # Add the relation tuple to the set of relations
            relations.add(relation_tuple)

# At this point, 'relations' contains all the relations between entities in the 'train_data'
print(len(relations))
print(relations)

3300 relations

In [ ]:
from py2neo import Graph, Node, Relationship

# 连接到Neo4j数据库
graph = Graph("bolt://localhost:7689", auth=("neo4j", "neo4j"))

# 创建节点
# alice = Node("Person", name="Alice", age=25)
# bob = Node("Person", name="Bob", age=30)
# graph.create(alice)
# graph.create(bob)

# # 创建关系
# alice_knows_bob = Relationship(alice, "KNOWS", bob)
# graph.create(alice_knows_bob)

# normalisation

In [ ]:
import torch
from transformers import BertTokenizer, BertModel

def get_bert_embedding(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    return last_hidden_states


def get_cls_embedding(text, model, tokenizer):
    embeddings = get_bert_embedding(text, model, tokenizer)
    cls_embedding = embeddings[0, 0, :]
    return cls_embedding

def get_first_word_embedding(text, model, tokenizer):
    embeddings = get_bert_embedding(text, model, tokenizer)
    first_word = text.split(' is ')[0]
    tokens = tokenizer.tokenize(first_word)
    word_embeddings = embeddings[0, 1:len(tokens)+1, :]
    mean_embedding = torch.mean(word_embeddings, dim=0)
    return mean_embedding

# Load pre-trained BERT model and tokenizer
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

text = "Pneumothorax is An accumulation of air or gas in the PLEURAL CAVITY, which may occur spontaneously or as a result of trauma or a pathological process. The gas may also be introduced deliberately during PNEUMOTHORAX, ARTIFICIAL."

cls_embedding = get_cls_embedding(text, model, tokenizer)
print(f"CLS Embedding shape: {cls_embedding.shape}")

first_word_embedding = get_first_word_embedding(text, model, tokenizer)  
print(f"First Word Embedding shape: {first_word_embedding.shape}")


from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def find_most_similar_item(mention_embedding, embeddings_array, items):
    # Calculate cosine similarity
    similarities = cosine_similarity(mention_embedding, embeddings_array)

    # Print the cosine similarity for each item
    for i, similarity in enumerate(similarities[0]):
        print(f"Cosine similarity with item {i+1}: {similarity:.4f}")

    # Find the index of the most similar item
    most_similar_index = similarities.argmax()
    print(f"\nMost similar item: \n{items[most_similar_index]}")

def find_nearest_item(mention_embedding, embeddings_array, items):
    # Calculate the Euclidean distance between mention_embedding and each embedding in embeddings_array
    distances = np.linalg.norm(mention_embedding - embeddings_array, axis=1)

    # Find the index of the nearest embedding
    nearest_index = np.argmin(distances)

    # Print the nearest item
    nearest_item = items[nearest_index]
    print(f"\nNearest item of Euclidean distance: \n{nearest_item}")

In [ ]:
# Define the items
item1 = "Pneumothorax is An accumulation of air or gas in the PLEURAL CAVITY, which may occur spontaneously or as a result of trauma or a pathological process. The gas may also be introduced deliberately during PNEUMOTHORAX, ARTIFICIAL. (MSH)"
item2 = "Lungs is Either of the pair of organs occupying the cavity of the thorax that effect the aeration of the blood. (MSH)"
item3 = "Pleural is Of or pertaining to the pleura. (NCI)"
item4 = "Thyroidectomy is Surgical removal of the thyroid gland." 
items = [item1, item2, item3, item4]



model_name = 'bert-base-uncased'
model_name = 'emilyalsentzer/Bio_ClinicalBERT'
model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)


# Encode the items using the CLS embedding
# embeddings = [get_first_word_embedding(item, model, tokenizer) for item in items]
embeddings = [get_cls_embedding(item, model, tokenizer) for item in items]
embeddings_array = torch.stack(embeddings).detach().numpy()


# Get embedding for the mention
mention = "Pneumothorax"  
mention_embedding = get_cls_embedding(mention, model, tokenizer).detach().numpy().reshape(1, -1)

print(f"Mention: {mention}\n")
find_most_similar_item(mention_embedding, embeddings_array, items)
find_nearest_item(mention_embedding, embeddings_array, items)

In [ ]:
# Get embedding for the mention
mention = "lung"  
mention_embedding = get_cls_embedding(mention, model, tokenizer).detach().numpy().reshape(1, -1)

print(f"Mention: {mention}\n")
find_most_similar_item(mention_embedding, embeddings_array, items)
find_nearest_item(mention_embedding, embeddings_array, items)

In [ ]:
# Get embedding for the mention
mention = "Pneumothorax is the buildup of air or gas in the pleural cavity, which can happen spontaneously, due to trauma, or because of a disease. The gas can also be intentionally introduced during artificial pneumothorax."  
mention_embedding = get_cls_embedding(mention, model, tokenizer).detach().numpy().reshape(1, -1)

print(f"Mention: {mention}\n")
find_most_similar_item(mention_embedding, embeddings_array, items)
find_nearest_item(mention_embedding, embeddings_array, items)

In [ ]:
# Get embedding for the mention
mention = "PLungs are one of the pair of organs located in the thoracic cavity that facilitate the aeration of blood."  
mention_embedding = get_cls_embedding(mention, model, tokenizer).detach().numpy().reshape(1, -1)

print(f"Mention: {mention}\n")
find_most_similar_item(mention_embedding, embeddings_array, items)
find_nearest_item(mention_embedding, embeddings_array, items)

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import re

# 示例文本
text = """苹果公司是一家科技巨头，总部位于加利福尼亚。它以iPhone和Mac电脑而闻名。
苹果公司在全球拥有众多粉丝，产品设计备受赞誉。微软是另一家主要的科技公司，
专注于软件开发。两家公司都在AI领域有重大投资。谷歌也是AI领域的重要玩家，
与苹果公司在多个领域展开竞争。"""

main_entity = "苹果公司"
related_entities = ["微软", "谷歌"]
attributes = ["科技", "创新", "产品"]

# 方法1: 实体中心化嵌入
def entity_centric_embedding(text, entity):
    return f"关于{entity}的信息：{text}"

# 方法2: 相关句子提取嵌入
def relevant_sentence_embedding(text, entity):
    sentences = sent_tokenize(text)
    relevant_sentences = [s for s in sentences if entity in s]
    return f"{entity}: {' '.join(relevant_sentences)}"

# 方法3: 实体上下文窗口嵌入
def context_window_embedding(text, entity, window_size=1):
    sentences = sent_tokenize(text)
    relevant_sentences = []
    for i, sentence in enumerate(sentences):
        if entity in sentence:
            start = max(0, i - window_size)
            end = min(len(sentences), i + window_size + 1)
            relevant_sentences.extend(sentences[start:end])
    return f"{entity} 上下文: {' '.join(relevant_sentences)}"

# 方法4: 实体属性强化嵌入
def entity_attribute_embedding(text, entity, attributes):
    pattern = r'\b' + re.escape(entity) + r'\b.{0,50}'
    matches = re.findall(pattern, text)
    attribute_text = " ".join([f"{entity} {attr}" for attr in attributes])
    context = " ".join(matches)
    return f"{attribute_text}\n实体上下文: {context}"

# 方法5: 实体关系焦点嵌入
def entity_relation_embedding(text, main_entity, related_entities):
    sentences = sent_tokenize(text)
    relevant_sentences = [s for s in sentences if main_entity in s and any(e in s for e in related_entities)]
    return f"{main_entity}与其他实体的关系: {' '.join(relevant_sentences)}"

# 打印每种方法的结果
print("方法1: 实体中心化嵌入")
print(entity_centric_embedding(text, main_entity))
print("\n方法2: 相关句子提取嵌入")
print(relevant_sentence_embedding(text, main_entity))
print("\n方法3: 实体上下文窗口嵌入")
print(context_window_embedding(text, main_entity))
print("\n方法4: 实体属性强化嵌入")
print(entity_attribute_embedding(text, main_entity, attributes))
print("\n方法5: 实体关系焦点嵌入")
print(entity_relation_embedding(text, main_entity, related_entities))